In [ ]:
!uv pip install neo4j

# Import / Config

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from neo4j import GraphDatabase
from dotenv import dotenv_values

c_pg = dotenv_values("cred-pg.txt")
c_neo4j = dotenv_values("cred-neo4j.txt")

## Postgres

In [ ]:
engine = create_engine(
    f'postgresql+psycopg://{c_pg["user"]}:{c_pg["password"]}@{c_pg["host"]}/{c_pg["dbname"]}', 
    connect_args = {
        'options': '-c search_path=${user},ugeobln,ugm,uinsta,umisc,umobility,usozmed,public', 
        'keepalives_idle': 120
    },
    pool_size=1, 
    max_overflow=0,
    execution_options={ 'isolation_level': 'AUTOCOMMIT' }
)

In [ ]:
with engine.connect() as con:
    haltestelle_pdf = pd.read_sql_query("select * from haltestelle", con)
    segment_pdf = pd.read_sql_query("select * from segment", con)
    linie_pdf = pd.read_sql_query("select * from linie", con)
    unterlinie_pdf = pd.read_sql_query("select * from unterlinie", con)
    abschnitt_pdf = pd.read_sql_query("select * from abschnitt", con)

In [ ]:
abschnitt_pdf.info()

## neo4j

In [ ]:
neo4j_host = c_neo4j["neo4j_host"]
neo4j_auth = (c_neo4j["neo4j_userid"], c_neo4j["neo4j_password"])
neo4j_host, neo4j_auth

In [ ]:
cypher_create_stop = 'CREATE (h:Haltestelle {hid: $hid, bez: $bez, lat:$lat, lng:$lng})'
with GraphDatabase.driver(neo4j_host, auth=neo4j_auth) as driver:
    with driver.session() as session:
        for r in haltestelle_pdf.itertuples(index=False):
            session.run(cypher_create_stop, hid=r.hid, bez=r.bez, lat=r.lat, lng=r.lng)

In [ ]:
cypher_create_linie = 'CREATE (l:Linie {lid: $lid, bez: $bez})'
with GraphDatabase.driver(neo4j_host, auth=neo4j_auth) as driver:
    with driver.session() as session:
        for r in linie_pdf.itertuples(index=False):
            session.run(cypher_create_linie, lid=r.lid, bez=r.bez)

In [ ]:
cypher_create_segment = '''
MATCH (ha:Haltestelle), (hb:Haltestelle)
WHERE ha.hid=$hid_a AND hb.hid=$hid_b
CREATE (s:Segment {hid_a: ha.hid, hid_b: hb.hid, laengeInMeter: $laengeInMeter})
CREATE (s) -[:ProjSegA]-> (ha)
CREATE (s) -[:ProjSegB]-> (hb)
'''
with GraphDatabase.driver(neo4j_host, auth=neo4j_auth) as driver:
    with driver.session() as session:
        for r in segment_pdf.itertuples(index=False):
            session.run(cypher_create_segment, hid_a=r.hid_a, hid_b=r.hid_b, laengeInMeter=r.laenge_in_meter)

In [ ]:
cypher_create_unterlinie = '''
MATCH (l:Linie)
WHERE l.lid=$lid
CREATE (ul:Unterlinie {ulid: $ulid})
CREATE (ul) -[:InL]-> (l)
'''
with GraphDatabase.driver(neo4j_host, auth=neo4j_auth) as driver:
    with driver.session() as session:
        for r in unterlinie_pdf.itertuples(index=False):
            session.run(cypher_create_unterlinie, ulid=r.ulid, lid=r.lid)

In [ ]:
cypher_create_abschnitt = '''
MATCH (ha:Haltestelle), (hb:Haltestelle), (ul:Unterlinie)
WHERE ha.hid=$hid_a AND hb.hid=$hid_b AND ul.ulid=$ulid
CREATE (a:Abschnitt {nr: $nr, haelt: $haelt})
CREATE (a) -[:InUL]-> (ul)
CREATE (a) -[:ProjAbA]-> (ha)
CREATE (a) -[:ProjAbB]-> (hb)
'''
with GraphDatabase.driver(neo4j_host, auth=neo4j_auth) as driver:
    with driver.session() as session:
        for r in abschnitt_pdf.itertuples(index=False):
            session.run(cypher_create_abschnitt, ulid=r.ulid, nr=r.nr, hid_a=r.hid_a, hid_b=r.hid_b, haelt=r.haelt)